# 🎯 AI Interview Coach with LangGraph

A comprehensive interview preparation system that simulates realistic interviews using multi-agent workflow orchestration.

**Workflow:** research_company → research_interviewer → generate_questions → interviewer_bot ↔ get_response → final_report

---

## 📦 Section 1: Setup & Dependencies

In [ ]:
# Install required packages
!pip install -q langgraph langchain langchain-openai langchain-anthropic tavily-python google-search-results python-dotenv

In [ ]:
# Import core libraries
import os
import json
import asyncio
from getpass import getpass
from typing import TypedDict, Annotated, Literal, Optional, List, Dict, Any
from datetime import datetime

# LangChain & LangGraph
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, SystemMessage

# Research APIs
from tavily import TavilyClient
from serpapi import GoogleSearch

# Colab utilities
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
    print("⚠️  Not running in Colab - file export features will be limited")

print("✅ All imports successful!")

## 🔑 Section 2: Configuration & API Keys

In [ ]:
# Configure API Keys (secure input)
print("🔐 Enter your API keys (input will be hidden)\n")

# LLM Provider Keys
OPENAI_API_KEY = getpass("OpenAI API Key (optional, press Enter to skip): ") or None
ANTHROPIC_API_KEY = getpass("Anthropic API Key (optional, press Enter to skip): ") or None

# Research API Keys
TAVILY_API_KEY = getpass("Tavily API Key (optional, press Enter to skip): ") or None
SERPAPI_KEY = getpass("SERP API Key (optional, press Enter to skip): ") or None

# Set environment variables
if OPENAI_API_KEY:
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
if ANTHROPIC_API_KEY:
    os.environ['ANTHROPIC_API_KEY'] = ANTHROPIC_API_KEY
if TAVILY_API_KEY:
    os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY
if SERPAPI_KEY:
    os.environ['SERPAPI_API_KEY'] = SERPAPI_KEY

print("\n✅ Configuration complete!")
print(f"   OpenAI: {'✓' if OPENAI_API_KEY else '✗'}")
print(f"   Anthropic: {'✓' if ANTHROPIC_API_KEY else '✗'}")
print(f"   Tavily: {'✓' if TAVILY_API_KEY else '✗'}")
print(f"   SERP API: {'✓' if SERPAPI_KEY else '✗'}")

In [ ]:
# LLM Provider Selection
LLM_PROVIDER = "anthropic"  # Options: "openai" or "anthropic"
MODEL_NAME = "claude-3-5-sonnet-20241022" if LLM_PROVIDER == "anthropic" else "gpt-4-turbo"

print(f"🤖 Using: {LLM_PROVIDER.title()} ({MODEL_NAME})")

# Initialize LLM
def get_llm(temperature=0.7):
    if LLM_PROVIDER == "anthropic":
        if not ANTHROPIC_API_KEY:
            raise ValueError("Anthropic API key not provided")
        return ChatAnthropic(model=MODEL_NAME, temperature=temperature)
    else:
        if not OPENAI_API_KEY:
            raise ValueError("OpenAI API key not provided")
        return ChatOpenAI(model=MODEL_NAME, temperature=temperature)

# Test LLM
try:
    llm = get_llm()
    print("✅ LLM initialized successfully")
except Exception as e:
    print(f"❌ LLM initialization failed: {e}")

## 📊 Section 3: State Schema Definition

In [ ]:
# Define the interview session state
class InterviewState(TypedDict):
    # Input configuration
    company_name: str
    role_title: str
    job_description: Optional[str]
    interviewer_name: Optional[str]
    candidate_background: Optional[str]
    seniority_level: Literal["junior", "mid", "senior", "lead"]
    interview_type: Literal["behavioral", "system-design", "coding", "mixed"]
    
    # Research outputs
    company_summary: Optional[str]
    talking_points: Optional[List[str]]
    interviewer_persona: Optional[str]
    interviewer_hypotheses: Optional[List[str]]
    
    # Question generation
    questions: List[str]
    
    # Interview session state
    conversation_history: List[Dict[str, Any]]
    current_question_index: int
    stop_requested: bool
    coach_mode_active: bool
    
    # Progress tracking
    coverage_tracker: Dict[str, float]
    
    # Final output
    final_report: Optional[str]

# Initialize default state
def create_initial_state(
    company_name: str,
    role_title: str,
    seniority_level: str = "mid",
    interview_type: str = "mixed",
    job_description: str = None,
    interviewer_name: str = None,
    candidate_background: str = None
) -> InterviewState:
    return {
        "company_name": company_name,
        "role_title": role_title,
        "job_description": job_description,
        "interviewer_name": interviewer_name,
        "candidate_background": candidate_background,
        "seniority_level": seniority_level,
        "interview_type": interview_type,
        "company_summary": None,
        "talking_points": [],
        "interviewer_persona": None,
        "interviewer_hypotheses": [],
        "questions": [],
        "conversation_history": [],
        "current_question_index": 0,
        "stop_requested": False,
        "coach_mode_active": False,
        "coverage_tracker": {
            "problem_solving": 0.0,
            "collaboration": 0.0,
            "ambiguity_handling": 0.0,
            "leadership_ownership": 0.0,
            "technical_depth": 0.0
        },
        "final_report": None
    }

print("✅ State schema defined")

## 🔍 Section 4: Research Utilities

In [ ]:
# Research utility functions
def search_with_tavily(query: str, max_results: int = 5) -> List[Dict[str, str]]:
    """Search using Tavily API"""
    if not TAVILY_API_KEY:
        return []
    
    try:
        client = TavilyClient(api_key=TAVILY_API_KEY)
        response = client.search(query, max_results=max_results)
        return response.get('results', [])
    except Exception as e:
        print(f"⚠️  Tavily search failed: {e}")
        return []

def search_with_serp(query: str, max_results: int = 5) -> List[Dict[str, str]]:
    """Search using SERP API as fallback"""
    if not SERPAPI_KEY:
        return []
    
    try:
        params = {
            "q": query,
            "api_key": SERPAPI_KEY,
            "num": max_results
        }
        search = GoogleSearch(params)
        results = search.get_dict()
        
        organic = results.get('organic_results', [])
        return [
            {"title": r.get("title", ""), "url": r.get("link", ""), "content": r.get("snippet", "")}
            for r in organic[:max_results]
        ]
    except Exception as e:
        print(f"⚠️  SERP API search failed: {e}")
        return []

def search_web(query: str, max_results: int = 5) -> List[Dict[str, str]]:
    """Search with Tavily, fallback to SERP"""
    print(f"🔍 Searching: {query}")
    
    results = search_with_tavily(query, max_results)
    
    if not results:
        print("  → Tavily failed, trying SERP API...")
        results = search_with_serp(query, max_results)
    
    if results:
        print(f"  ✓ Found {len(results)} results")
    else:
        print("  ✗ No results found")
    
    return results

print("✅ Research utilities ready")

## 🏢 Section 5: Node - Research Company

In [ ]:
def research_company_node(state: InterviewState) -> Dict[str, Any]:
    """Research company background, tech stack, and culture"""
    print("\n" + "="*60)
    print("🏢 RESEARCHING COMPANY")
    print("="*60)
    
    company = state['company_name']
    role = state['role_title']
    
    # Multi-faceted search
    searches = [
        f"{company} products technology stack {role}",
        f"{company} recent news latest developments 2025 2026",
        f"{company} company culture values engineering"
    ]
    
    all_results = []
    for query in searches:
        results = search_web(query, max_results=3)
        all_results.extend(results)
    
    # Synthesize with LLM
    if all_results:
        context = "\n\n".join([
            f"Title: {r.get('title', '')}\nContent: {r.get('content', '')}" 
            for r in all_results
        ])
    else:
        context = "No search results available. Use general knowledge."
    
    llm = get_llm(temperature=0.3)
    
    prompt = f"""You are a senior career coach preparing a candidate for an interview.

Company: {company}
Role: {role}

Research context:
{context}

Create a concise company summary (200-300 words) covering:
1. Industry & business model
2. Key products/services
3. Technology stack (if relevant to {role})
4. Recent news or developments
5. Culture & values signals

Then provide 5-8 specific talking points the candidate should be ready to reference.

Format:
# Company Summary
[summary here]

# Talking Points
- [point 1]
- [point 2]
...
"""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    output = response.content
    
    # Parse output
    parts = output.split("# Talking Points")
    summary = parts[0].replace("# Company Summary", "").strip()
    
    talking_points = []
    if len(parts) > 1:
        points_text = parts[1].strip()
        talking_points = [
            line.strip("- ").strip() 
            for line in points_text.split("\n") 
            if line.strip().startswith("-")
        ]
    
    print("\n📄 Summary:")
    print(summary[:200] + "...")
    print(f"\n💡 Generated {len(talking_points)} talking points")
    
    return {
        "company_summary": summary,
        "talking_points": talking_points
    }

print("✅ Company research node ready")

## 👤 Section 6: Node - Research Interviewer

In [ ]:
def research_interviewer_node(state: InterviewState) -> Dict[str, Any]:
    """Research interviewer background and create persona"""
    print("\n" + "="*60)
    print("👤 RESEARCHING INTERVIEWER")
    print("="*60)
    
    interviewer = state.get('interviewer_name') or "Generic interviewer"
    company = state['company_name']
    role = state['role_title']
    
    all_results = []
    
    if interviewer and interviewer.lower() not in ["generic", "unknown", "na", "n/a", ""]:
        # Search for specific interviewer
        searches = [
            f"{interviewer} {company} LinkedIn",
            f"{interviewer} GitHub publications talks",
            f"{interviewer} {company} engineering"
        ]
        
        for query in searches:
            results = search_web(query, max_results=2)
            all_results.extend(results)
    
    # Synthesize persona
    if all_results:
        context = "\n\n".join([
            f"Title: {r.get('title', '')}\nContent: {r.get('content', '')}" 
            for r in all_results
        ])
    else:
        context = f"No specific information found. Infer a realistic persona for a {role} interviewer at {company}."
    
    llm = get_llm(temperature=0.4)
    
    prompt = f"""You are a senior career coach analyzing an interviewer profile.

Interviewer: {interviewer}
Company: {company}
Role being interviewed for: {role}

Research context:
{context}

Create:
1. A concise persona description (50-100 words): likely title, team, background, interests
2. 5-7 specific hypotheses about what this interviewer will care about

Format each hypothesis as:
- "Likely to probe [specific area] because [reason]" OR
- "Red flag if candidate doesn't mention [specific thing]"

Format:
# Persona
[description]

# Hypotheses
- [hypothesis 1]
- [hypothesis 2]
...
"""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    output = response.content
    
    # Parse output
    parts = output.split("# Hypotheses")
    persona = parts[0].replace("# Persona", "").strip()
    
    hypotheses = []
    if len(parts) > 1:
        hypotheses_text = parts[1].strip()
        hypotheses = [
            line.strip("- ").strip() 
            for line in hypotheses_text.split("\n") 
            if line.strip().startswith("-")
        ]
    
    print("\n📋 Persona:")
    print(persona)
    print(f"\n🎯 Generated {len(hypotheses)} interviewer hypotheses")
    
    return {
        "interviewer_persona": persona,
        "interviewer_hypotheses": hypotheses
    }

print("✅ Interviewer research node ready")

## ❓ Section 7: Node - Generate Questions

In [ ]:
def generate_questions_node(state: InterviewState) -> Dict[str, Any]:
    """Generate targeted interview questions"""
    print("\n" + "="*60)
    print("❓ GENERATING INTERVIEW QUESTIONS")
    print("="*60)
    
    # Seniority-specific instructions
    seniority_guide = {
        "junior": "Focus on learning agility, collaboration, foundational technical knowledge, and growth mindset.",
        "mid": "Balance technical depth with ownership, cross-team collaboration, and independent problem-solving.",
        "senior": "Emphasize system design, trade-offs, impact metrics, mentorship, and handling ambiguity.",
        "lead": "Prioritize architectural decisions, organizational impact, technical leadership, and strategic thinking."
    }
    
    seniority = state['seniority_level']
    interview_type = state['interview_type']
    
    # Build context
    company_context = state.get('company_summary', 'No company summary available')
    interviewer_context = state.get('interviewer_persona', 'Generic interviewer')
    job_desc = state.get('job_description', 'Not provided')
    candidate_bg = state.get('candidate_background', 'Not provided')
    
    llm = get_llm(temperature=0.8)
    
    prompt = f"""You are an expert interview question designer for {state['role_title']} at {state['company_name']}.

CONTEXT:
Company: {company_context}

Interviewer: {interviewer_context}

Seniority: {seniority} - {seniority_guide[seniority]}

Interview Type: {interview_type}

Job Description: {job_desc}

Candidate Background: {candidate_bg}

TASK:
Generate 10-15 interview questions following this distribution:
- 40-60% behavioral (STAR-style questions about past experiences)
- 20-40% role-specific technical (system design, coding concepts, architecture)
- At least 2 "curveball" or meta questions (trade-offs, failure stories, ethical dilemmas)

REQUIREMENTS:
✓ Tie questions to the company's domain and tech stack when possible
✓ Match the interviewer's likely concerns and background
✓ Calibrate difficulty to {seniority} level
✓ Avoid generic questions - make them specific and targeted
✓ Ensure diversity - no repetitive patterns

Format as numbered list:
1. [Question text]
2. [Question text]
...
"""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    output = response.content
    
    # Parse numbered questions
    questions = []
    for line in output.split("\n"):
        line = line.strip()
        # Match patterns like "1." or "1)" at start
        if line and len(line) > 3:
            if line[0].isdigit() and line[1:3] in ['. ', ') ']:
                question = line.split('. ', 1)[-1] if '. ' in line else line.split(') ', 1)[-1]
                questions.append(question.strip())
    
    print(f"\n✨ Generated {len(questions)} questions")
    print("\nFirst 3 questions:")
    for i, q in enumerate(questions[:3], 1):
        print(f"  {i}. {q[:80]}...")
    
    return {"questions": questions}

print("✅ Question generation node ready")

## 🎤 Section 8: Node - Interviewer Bot

In [ ]:
def interviewer_bot_node(state: InterviewState) -> Dict[str, Any]:
    """Ask questions and provide feedback"""
    print("\n" + "="*60)
    print("🎤 INTERVIEWER BOT")
    print("="*60)
    
    questions = state['questions']
    current_idx = state['current_question_index']
    conversation = state['conversation_history']
    coverage = state['coverage_tracker'].copy()
    
    # Check if last turn needs feedback
    if conversation and conversation[-1].get('answer') and not conversation[-1].get('score'):
        print("\n📊 Analyzing your answer...")
        
        last_entry = conversation[-1]
        question = last_entry['question']
        answer = last_entry['answer']
        
        # Check for coach mode request
        coach_triggers = ['how could i', 'better structure', 'feedback on', 'improve this', 'tell me how']
        is_coach_request = any(trigger in answer.lower() for trigger in coach_triggers)
        
        llm = get_llm(temperature=0.4)
        
        if is_coach_request or state.get('coach_mode_active', False):
            # Coach mode - metacognitive feedback
            prompt = f"""You are a supportive interview coach in coaching mode.

Question: {question}
Candidate's response: {answer}

The candidate is asking for structural guidance. Provide:
1. What they did well
2. Specific suggestions for improving their answer structure (use STAR if applicable)
3. What additional details would strengthen the impact

Be encouraging and specific. Don't score - just coach.
"""
            response = llm.invoke([HumanMessage(content=prompt)])
            feedback = response.content
            
            print("\n💬 COACH FEEDBACK:")
            print(feedback)
            print("\n[Returning to interviewer mode for next question]\n")
            
            conversation[-1]['coach_feedback'] = feedback
            
            return {
                "conversation_history": conversation,
                "coach_mode_active": False
            }
        else:
            # Normal interviewer mode - score and rewrite
            prompt = f"""You are an experienced interviewer evaluating a candidate's answer.

Question: {question}
Candidate's answer: {answer}
Role: {state['role_title']}
Seniority: {state['seniority_level']}

Provide two sections:

## Score & Signal
- Score: [0-5]
- What this signals to an experienced interviewer
- Impact on hire confidence (increases/decreases/neutral)

## Rewrite to A+
A concise, improved version using STAR format where applicable, preserving the candidate's actual experience but structuring it optimally.
"""
            
            response = llm.invoke([HumanMessage(content=prompt)])
            feedback = response.content
            
            # Parse score
            score = 3  # default
            if "Score:" in feedback:
                try:
                    score_line = [l for l in feedback.split("\n") if "Score:" in l][0]
                    score = int(score_line.split("Score:")[1].strip()[0])
                except:
                    pass
            
            # Parse sections
            parts = feedback.split("## Rewrite to A+")
            signal = parts[0].replace("## Score & Signal", "").strip()
            rewrite = parts[1].strip() if len(parts) > 1 else "N/A"
            
            print("\n📊 SCORE & SIGNAL:")
            print(signal)
            print("\n✨ REWRITE TO A+:")
            print(rewrite)
            
            # Update coverage tracker
            coverage_prompt = f"""Analyze this interview answer and rate 0-100 how much it demonstrates each dimension:

Answer: {answer}

Dimensions:
- problem_solving
- collaboration
- ambiguity_handling
- leadership_ownership
- technical_depth

Return ONLY a JSON object like: {{"problem_solving": 60, "collaboration": 20, ...}}
"""
            
            try:
                cov_response = llm.invoke([HumanMessage(content=coverage_prompt)])
                cov_text = cov_response.content.strip()
                # Extract JSON from markdown code blocks if present
                if "```" in cov_text:
                    cov_text = cov_text.split("```")[1].replace("json", "").strip()
                cov_update = json.loads(cov_text)
                # Update coverage (take max)
                for dim, val in cov_update.items():
                    if dim in coverage:
                        coverage[dim] = max(coverage[dim], val / 100.0)
            except Exception as e:
                print(f"  ⚠️  Coverage update failed: {e}")
            
            conversation[-1].update({
                'score': score,
                'signal': signal,
                'rewrite': rewrite
            })
    
    # Ask next question
    if current_idx < len(questions):
        next_question = questions[current_idx]
        print(f"\n\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        print(f"Question {current_idx + 1}/{len(questions)}:")
        print(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        print(f"\n{next_question}\n")
    else:
        # Generate follow-up based on shallow answers
        shallow_answers = [c for c in conversation if c.get('score', 5) < 3]
        
        if shallow_answers:
            print("\n🔄 Generating follow-up question...")
            llm = get_llm(temperature=0.7)
            
            context = "\n".join([
                f"Q: {a['question']}\nA: {a['answer'][:100]}..." 
                for a in shallow_answers[-2:]
            ])
            
            prompt = f"""Based on these shallow/incomplete answers, generate ONE probing follow-up question:

{context}

Generate a question that digs deeper into an area the candidate avoided or glossed over.
"""
            response = llm.invoke([HumanMessage(content=prompt)])
            next_question = response.content.strip()
            
            print(f"\n\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
            print(f"Follow-up Question:")
            print(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
            print(f"\n{next_question}\n")
        else:
            next_question = None
    
    if next_question:
        conversation.append({
            'question': next_question,
            'answer': None,
            'score': None,
            'signal': None,
            'rewrite': None
        })
        
        return {
            "conversation_history": conversation,
            "current_question_index": current_idx + 1,
            "coverage_tracker": coverage
        }
    else:
        # No more questions - signal completion
        return {
            "conversation_history": conversation,
            "current_question_index": current_idx + 1,
            "coverage_tracker": coverage,
            "stop_requested": True
        }

print("✅ Interviewer bot node ready")

## 💬 Section 9: Node - Get Response

In [ ]:
def get_response_node(state: InterviewState) -> Dict[str, Any]:
    """Capture user's answer"""
    conversation = state['conversation_history']
    
    if not conversation or conversation[-1].get('answer'):
        # No pending question
        return {}
    
    print("\n" + "─"*60)
    print("Your answer (type 'stop', 'end', or 'finish' to conclude):")
    print("─"*60)
    
    user_input = input("\n→ ")
    
    # Check for stop tokens
    stop_tokens = ['stop', 'end', 'finish', 'quit', 'exit']
    if user_input.lower().strip() in stop_tokens:
        print("\n🛑 Interview concluded by user.")
        return {"stop_requested": True}
    
    # Record answer
    conversation[-1]['answer'] = user_input
    
    return {
        "conversation_history": conversation
    }

print("✅ Get response node ready")

## 🔀 Section 10: Graph Construction

In [ ]:
# Build the workflow graph
def should_continue(state: InterviewState) -> Literal["get_response", "__end__"]:
    """Route from interviewer_bot: continue or end"""
    if state.get('stop_requested', False):
        return "__end__"
    
    # Also end if we've done 20+ questions
    if state.get('current_question_index', 0) > 20:
        return "__end__"
    
    # Check if there's a pending question
    conversation = state.get('conversation_history', [])
    if conversation and conversation[-1].get('answer') is None:
        return "get_response"
    
    return "__end__"

# Create graph
workflow = StateGraph(InterviewState)

# Add nodes
workflow.add_node("research_company", research_company_node)
workflow.add_node("research_interviewer", research_interviewer_node)
workflow.add_node("generate_questions", generate_questions_node)
workflow.add_node("interviewer_bot", interviewer_bot_node)
workflow.add_node("get_response", get_response_node)

# Add edges
workflow.add_edge(START, "research_company")
workflow.add_edge("research_company", "research_interviewer")
workflow.add_edge("research_interviewer", "generate_questions")
workflow.add_edge("generate_questions", "interviewer_bot")

# Conditional routing from interviewer_bot
workflow.add_conditional_edges(
    "interviewer_bot",
    should_continue,
    {
        "get_response": "get_response",
        "__end__": END
    }
)

# Loop back from get_response
workflow.add_edge("get_response", "interviewer_bot")

# Compile with checkpointing
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

print("✅ LangGraph workflow compiled!")
print("\n📊 Workflow structure:")
print("   START → research_company → research_interviewer → generate_questions")
print("         → interviewer_bot ↔ get_response → END")

## 📝 Section 11: Final Report Generation

In [ ]:
def generate_final_report(state: InterviewState) -> str:
    """Generate comprehensive final report"""
    print("\n" + "="*60)
    print("📝 GENERATING FINAL REPORT")
    print("="*60)
    
    conversation = state['conversation_history']
    coverage = state['coverage_tracker']
    
    # Build conversation summary
    convo_summary = "\n\n".join([
        f"Q: {c['question']}\nA: {c.get('answer', 'N/A')[:200]}...\nScore: {c.get('score', 'N/A')}" 
        for c in conversation[:10]  # Limit context
    ])
    
    coverage_summary = "\n".join([
        f"- {dim.replace('_', ' ').title()}: {int(score*100)}%" 
        for dim, score in coverage.items()
    ])
    
    llm = get_llm(temperature=0.5)
    
    prompt = f"""You are a senior interview coach providing a final debrief.

Interview for: {state['role_title']} at {state['company_name']}
Seniority: {state['seniority_level']}
Questions asked: {len(conversation)}

Coverage achieved:
{coverage_summary}

Sample conversation:
{convo_summary}

Generate a comprehensive final report with:

## 💪 Strengths
3-5 bullet points of what the candidate did well

## ⚠️  Risk Areas
2-4 bullet points of areas that need improvement

## 📚 Stories to Refine
3-5 specific examples/stories the candidate should polish, with what's missing (STAR gaps)

## 🎯 3-Session Practice Plan
Week-by-week focus areas:
- Session 1: [focus]
- Session 2: [focus]
- Session 3: [focus]

Be specific, actionable, and encouraging.
"""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    report = response.content
    
    return report

print("✅ Final report generator ready")

## 💾 Section 12: Export/Import Utilities

In [ ]:
def export_session(state: InterviewState, filename: str = None):
    """Export session state to JSON"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"interview_session_{timestamp}.json"
    
    # Convert state to JSON-serializable format
    export_data = dict(state)
    
    with open(filename, 'w') as f:
        json.dump(export_data, f, indent=2)
    
    print(f"\n💾 Session exported to: {filename}")
    
    if IN_COLAB:
        files.download(filename)
        print("   File download started...")
    
    return filename

def import_session(filename: str) -> InterviewState:
    """Import session state from JSON"""
    with open(filename, 'r') as f:
        state = json.load(f)
    
    print(f"✅ Session imported from: {filename}")
    return state

print("✅ Export/import utilities ready")

## 🚀 Section 13: Main Execution

### Initialize Interview Session

In [ ]:
# User inputs
print("🎯 Interview Setup\n")
print("="*60)

COMPANY_NAME = input("Company name: ").strip() or "Google"
ROLE_TITLE = input("Role title: ").strip() or "Senior Software Engineer"
INTERVIEWER_NAME = input("Interviewer name (optional): ").strip() or None

print("\nSeniority level: 1) Junior  2) Mid  3) Senior  4) Lead")
seniority_choice = input("Choose (1-4): ").strip() or "3"
SENIORITY_LEVEL = {"1": "junior", "2": "mid", "3": "senior", "4": "lead"}.get(seniority_choice, "mid")

print("\nInterview type: 1) Behavioral  2) System Design  3) Coding  4) Mixed")
type_choice = input("Choose (1-4): ").strip() or "4"
INTERVIEW_TYPE = {"1": "behavioral", "2": "system-design", "3": "coding", "4": "mixed"}.get(type_choice, "mixed")

JOB_DESCRIPTION = input("\nJob description (optional, paste or skip): ").strip() or None
CANDIDATE_BACKGROUND = input("Your background summary (optional): ").strip() or None

print("\n" + "="*60)
print("✅ Configuration complete!")
print(f"   Company: {COMPANY_NAME}")
print(f"   Role: {ROLE_TITLE}")
print(f"   Seniority: {SENIORITY_LEVEL}")
print(f"   Type: {INTERVIEW_TYPE}")
print("="*60)

### Start Interview

In [ ]:
# Create initial state
initial_state = create_initial_state(
    company_name=COMPANY_NAME,
    role_title=ROLE_TITLE,
    seniority_level=SENIORITY_LEVEL,
    interview_type=INTERVIEW_TYPE,
    job_description=JOB_DESCRIPTION,
    interviewer_name=INTERVIEWER_NAME,
    candidate_background=CANDIDATE_BACKGROUND
)

# Run workflow with thread for checkpointing
config = {"configurable": {"thread_id": "interview_session_1"}}

print("\n🚀 Starting interview workflow...\n")

# Execute interview loop
final_state = None
for output in app.stream(initial_state, config):
    # Stream processes each node; output is dict with node name as key
    pass

# Get final state from last checkpoint
final_state = app.get_state(config).values

print("\n" + "="*60)
print("🎉 INTERVIEW COMPLETE")
print("="*60)

### Generate & Display Final Report

In [ ]:
if final_state:
    final_report = generate_final_report(final_state)
    
    print("\n" + "="*60)
    print("📊 FINAL REPORT")
    print("="*60)
    print("\n" + final_report)
    print("\n" + "="*60)
    
    # Update state with report
    final_state['final_report'] = final_report

### Export Session (Optional)

In [ ]:
# Export the session
if final_state:
    export_choice = input("\nExport session to JSON? (y/n): ").strip().lower()
    if export_choice == 'y':
        export_session(final_state)
    else:
        print("\n📝 Session not exported. You can export later by calling export_session(final_state)")

## 📊 Section 14: Session Analytics (Optional)

In [ ]:
# Display session statistics
if final_state:
    print("\n📈 SESSION STATISTICS")
    print("="*60)
    
    conversation = final_state.get('conversation_history', [])
    coverage = final_state.get('coverage_tracker', {})
    
    # Question stats
    answered = [c for c in conversation if c.get('answer')]
    scored = [c for c in answered if c.get('score') is not None]
    
    print(f"\nQuestions:")
    print(f"  Total generated: {len(final_state.get('questions', []))}")
    print(f"  Asked: {len(conversation)}")
    print(f"  Answered: {len(answered)}")
    
    if scored:
        avg_score = sum(c['score'] for c in scored) / len(scored)
        print(f"  Average score: {avg_score:.1f}/5")
    
    print(f"\nCoverage:")
    for dim, score in coverage.items():
        bar_length = int(score * 20)
        bar = "█" * bar_length + "░" * (20 - bar_length)
        print(f"  {dim.replace('_', ' ').title():25} [{bar}] {int(score*100)}%")
    
    print("\n" + "="*60)

---

## 🎓 Usage Instructions

### Quick Start:
1. Run all cells in order (Runtime → Run all)
2. Enter API keys when prompted
3. Configure your interview in Section 13
4. Answer questions interactively
5. Type `stop` to end early, or complete all questions
6. Review your final report

### Tips:
- Use STAR format (Situation, Task, Action, Result) for behavioral questions
- Type variations of "how could I improve this" to trigger coach mode during any answer
- Export your session to track progress across multiple practice sessions
- Review the "Rewrite to A+" sections to see optimal answer structure

### Customization:
- Modify `LLM_PROVIDER` and `MODEL_NAME` in Section 2 to switch between providers
- Adjust question count in `generate_questions_node` prompt
- Customize seniority guidance in Section 7

---